In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import polars as pl

DATA_DIR = '/content/drive/My Drive/0_cursos/EOI Analitica Escalable/COLAB_NOTEBOOKS/DATA'

os.makedirs(DATA_DIR, exist_ok=True)

In [3]:
os.listdir(DATA_DIR)

['10882.csv',
 'airbnb_listings.parquet',
 'fragments_example.parquet',
 '0504030000213.csv']

In [4]:
!pip install pyspark
!apt-get install openjdk-18-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-18-openjdk-amd64"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"

In [5]:
!java --version
!/usr/bin/java --version
!whereis java

openjdk 18.0.2-ea 2022-07-19
OpenJDK Runtime Environment (build 18.0.2-ea+9-Ubuntu-222.04)
OpenJDK 64-Bit Server VM (build 18.0.2-ea+9-Ubuntu-222.04, mixed mode, sharing)
openjdk 18.0.2-ea 2022-07-19
OpenJDK Runtime Environment (build 18.0.2-ea+9-Ubuntu-222.04)
OpenJDK 64-Bit Server VM (build 18.0.2-ea+9-Ubuntu-222.04, mixed mode, sharing)
java: /usr/bin/java /usr/share/java /usr/share/man/man1/java.1.gz


In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sqlf

spark = (SparkSession.builder
    .appName("Proyecto")
    .master("local[*]")
    .getOrCreate())
spark

In [7]:
# Lectura de datos
airbnb_spark = spark.read.parquet(DATA_DIR + "/airbnb_listings.parquet")
airbnb_polars = pl.read_parquet(DATA_DIR + "/airbnb_listings.parquet")

In [8]:
# Esquemas
print("=== Esquema Airbnb (Spark) ===")
airbnb_spark.printSchema()

print("=== Esquema Airbnb (Polars) ===")
print(airbnb_polars.schema)

=== Esquema Airbnb (Spark) ===
root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: long (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: long (nullable = true)
 |-- number_of_reviews: long (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: long (nullable = true)
 |-- availability_365: long (nullable = true)
 |-- number_of_reviews_ltm: long (nullable = true)
 |-- license: string (nullable = true)

=== Esquema Airbnb (Polars) ===
Schema([('id', Int64), ('name', String), ('host_id', Int64), ('host_name', String), ('neighbourhood_group', String), ('neighbourhood', String), ('latitud

In [9]:
# 1. Distribución de Precios

price_distribution_spark = (
    airbnb_spark.groupBy("room_type")
    .agg(
        sqlf.count("*").alias("count"),
        sqlf.round(sqlf.mean("price"), 2).alias("mean_price"),
        sqlf.min("price").alias("min_price"),
        sqlf.max("price").alias("max_price"),
        sqlf.percentile("price", 0.5).alias("median_price")
    )
)

price_distribution_spark.show()

+---------------+-----+----------+---------+---------+------------+
|      room_type|count|mean_price|min_price|max_price|median_price|
+---------------+-----+----------+---------+---------+------------+
|    Shared room|  343|    100.05|     13.0|   1500.0|        37.0|
|     Hotel room|   92|    303.66|     31.0|   1029.0|       161.0|
|Entire home/apt|17490|    169.14|      1.0|  21000.0|       124.0|
|   Private room| 9055|     72.76|      8.0|   9218.0|        45.0|
+---------------+-----+----------+---------+---------+------------+



In [10]:
# 1. Distribución de Precios

price_distribution_polars = (
    airbnb_polars.group_by("room_type")
    .agg(
        pl.col("price").count().alias("count"),
        pl.col("price").mean().round(2).alias("mean_price"),
        pl.col("price").min().alias("min_price"),
        pl.col("price").max().alias("max_price"),
        pl.col("price").median().alias("median_price")
    )
)


price_distribution_polars

room_type,count,mean_price,min_price,max_price,median_price
str,u32,f64,f64,f64,f64
"""Entire home/apt""",14622,169.14,1.0,21000.0,124.0
"""Private room""",6110,72.76,8.0,9218.0,45.0
"""Shared room""",283,100.05,13.0,1500.0,37.0
"""Hotel room""",53,303.66,31.0,1029.0,161.0


In [11]:
# 2. Análisis Geográfico

neighbourhood_prices_spark = (
    airbnb_spark.groupBy("neighbourhood")
    .agg(
        sqlf.round(sqlf.mean("price"), 2).alias("mean_price")
    )
    .orderBy(sqlf.desc("mean_price"))
)


expensive_neighbourhood = neighbourhood_prices_spark.limit(1)
cheapest_neighbourhood = neighbourhood_prices_spark.orderBy("mean_price").limit(1)

expensive_neighbourhood.show()
cheapest_neighbourhood.show()

+---------------+----------+
|  neighbourhood|mean_price|
+---------------+----------+
|Palomeras Bajas|    404.27|
+---------------+----------+

+-------------+----------+
|neighbourhood|mean_price|
+-------------+----------+
|    Vinateros|     36.29|
+-------------+----------+



In [12]:
# 2. Análisis Geográfico

lat_quintiles = airbnb_spark.approxQuantile("latitude", [0.2, 0.4, 0.6, 0.8], 0.0)
long_quintiles = airbnb_spark.approxQuantile("longitude", [0.2, 0.4, 0.6, 0.8], 0.0)


price_by_geo_spark = (
    airbnb_spark
    .withColumn("lat_quintile",
                sqlf.when(sqlf.col("latitude") <= lat_quintiles[0], 1)
                .when(sqlf.col("latitude") <= lat_quintiles[1], 2)
                .when(sqlf.col("latitude") <= lat_quintiles[2], 3)
                .when(sqlf.col("latitude") <= lat_quintiles[3], 4)
                .otherwise(5))
    .withColumn("long_quintile",
                sqlf.when(sqlf.col("longitude") <= long_quintiles[0], 1)
                .when(sqlf.col("longitude") <= long_quintiles[1], 2)
                .when(sqlf.col("longitude") <= long_quintiles[2], 3)
                .when(sqlf.col("longitude") <= long_quintiles[3], 4)
                .otherwise(5))
    .groupBy("lat_quintile", "long_quintile")
    .agg(
        sqlf.round(sqlf.mean("price"), 2).alias("mean_price"),
        sqlf.count("*").alias("count")
    )
    .orderBy(sqlf.desc("mean_price"))
)

price_by_geo_spark.show()

+------------+-------------+----------+-----+
|lat_quintile|long_quintile|mean_price|count|
+------------+-------------+----------+-----+
|           3|            5|    208.79|  346|
|           3|            1|    208.72|  779|
|           3|            4|    194.25|  814|
|           4|            4|    178.57|  959|
|           5|            2|     173.3|  554|
|           5|            4|    172.45| 1512|
|           3|            2|    170.56| 1801|
|           3|            3|    164.87| 1659|
|           4|            2|    151.61| 1068|
|           4|            1|    146.79|  819|
|           2|            1|    145.48| 1394|
|           4|            3|    145.42|  952|
|           2|            2|    143.86| 1551|
|           2|            4|    137.01|  810|
|           5|            1|    125.93|  674|
|           4|            5|    125.76| 1595|
|           2|            3|    125.49| 1487|
|           5|            5|    118.24| 1884|
|           1|            3|    11

In [13]:
# 2. Análisis Geográfico

neighbourhood_prices_polars = (
    airbnb_polars.group_by("neighbourhood")
    .agg(
        pl.col("price").mean().round(2).alias("mean_price")
    )
    .sort("mean_price", descending=True)
)


expensive_neighbourhood = neighbourhood_prices_polars.head(1)
cheapest_neighbourhood = neighbourhood_prices_polars.tail(1)

print(expensive_neighbourhood)

print(cheapest_neighbourhood)

shape: (1, 2)
┌─────────────────┬────────────┐
│ neighbourhood   ┆ mean_price │
│ ---             ┆ ---        │
│ str             ┆ f64        │
╞═════════════════╪════════════╡
│ Palomeras Bajas ┆ 404.27     │
└─────────────────┴────────────┘
shape: (1, 2)
┌───────────────┬────────────┐
│ neighbourhood ┆ mean_price │
│ ---           ┆ ---        │
│ str           ┆ f64        │
╞═══════════════╪════════════╡
│ Vinateros     ┆ 36.29      │
└───────────────┴────────────┘


In [14]:
# 2. Análisis Geográfico


lat_quintiles = [
    airbnb_polars["latitude"].quantile(0.2),
    airbnb_polars["latitude"].quantile(0.4),
    airbnb_polars["latitude"].quantile(0.6),
    airbnb_polars["latitude"].quantile(0.8)
]

long_quintiles = [
    airbnb_polars["longitude"].quantile(0.2),
    airbnb_polars["longitude"].quantile(0.4),
    airbnb_polars["longitude"].quantile(0.6),
    airbnb_polars["longitude"].quantile(0.8)
]


price_by_geo_polars = (
    airbnb_polars
    .with_columns([
        pl.when(pl.col("latitude") <= lat_quintiles[0]).then(1)
        .when(pl.col("latitude") <= lat_quintiles[1]).then(2)
        .when(pl.col("latitude") <= lat_quintiles[2]).then(3)
        .when(pl.col("latitude") <= lat_quintiles[3]).then(4)
        .otherwise(5).alias("lat_quintile"),

        pl.when(pl.col("longitude") <= long_quintiles[0]).then(1)
        .when(pl.col("longitude") <= long_quintiles[1]).then(2)
        .when(pl.col("longitude") <= long_quintiles[2]).then(3)
        .when(pl.col("longitude") <= long_quintiles[3]).then(4)
        .otherwise(5).alias("long_quintile"),
    ])
    .group_by("lat_quintile", "long_quintile")
    .agg(
        pl.col("price").mean().round(2).alias("mean_price"),
        pl.col("price").count().alias("count")
    )
    .sort("mean_price", descending=True)
)

print(price_by_geo_polars)

shape: (25, 4)
┌──────────────┬───────────────┬────────────┬───────┐
│ lat_quintile ┆ long_quintile ┆ mean_price ┆ count │
│ ---          ┆ ---           ┆ ---        ┆ ---   │
│ i32          ┆ i32           ┆ f64        ┆ u32   │
╞══════════════╪═══════════════╪════════════╪═══════╡
│ 3            ┆ 5             ┆ 209.22     ┆ 264   │
│ 3            ┆ 1             ┆ 208.72     ┆ 621   │
│ 3            ┆ 4             ┆ 194.25     ┆ 654   │
│ 4            ┆ 4             ┆ 178.57     ┆ 737   │
│ 5            ┆ 2             ┆ 173.3      ┆ 433   │
│ …            ┆ …             ┆ …          ┆ …     │
│ 1            ┆ 4             ┆ 115.4      ┆ 966   │
│ 1            ┆ 5             ┆ 102.34     ┆ 1095  │
│ 2            ┆ 5             ┆ 91.38      ┆ 117   │
│ 1            ┆ 1             ┆ 78.59      ┆ 1349  │
│ 1            ┆ 2             ┆ 73.91      ┆ 340   │
└──────────────┴───────────────┴────────────┴───────┘


In [15]:
# 3. Actividad de los Anfitriones

correlation = airbnb_spark.stat.corr("reviews_per_month", "availability_365")
print(f"Correlación: {round(correlation, 4)}")

Correlación: 0.0644


In [16]:
# 3. Actividad de los Anfitriones

reviews_quintiles = airbnb_spark.approxQuantile("reviews_per_month", [0.2, 0.4, 0.6, 0.8], 0.0)


grouped_by_quintiles_spark = (
    airbnb_spark
    .withColumn("reviews_quintile",
                sqlf.when(sqlf.col("reviews_per_month") <= reviews_quintiles[0], 1)
                .when(sqlf.col("reviews_per_month") <= reviews_quintiles[1], 2)
                .when(sqlf.col("reviews_per_month") <= reviews_quintiles[2], 3)
                .when(sqlf.col("reviews_per_month") <= reviews_quintiles[3], 4)
                .otherwise(5))
    .groupBy("reviews_quintile")
    .agg(
        sqlf.round(sqlf.mean("availability_365"), 2).alias("mean_availability"),
        sqlf.count("*").alias("count")
    )
    .orderBy("reviews_quintile")
)

grouped_by_quintiles_spark.show()

+----------------+-----------------+-----+
|reviews_quintile|mean_availability|count|
+----------------+-----------------+-----+
|               1|           113.52| 4332|
|               2|           164.42| 4212|
|               3|           175.34| 4270|
|               4|           172.24| 4217|
|               5|           156.32| 9949|
+----------------+-----------------+-----+



In [17]:
# 3. Actividad de los Anfitriones

correlation_polars = airbnb_polars.select(
    pl.corr("reviews_per_month", "availability_365").round(4)
)

print(f"Correlación: {correlation_polars}")

Correlación: shape: (1, 1)
┌───────────────────┐
│ reviews_per_month │
│ ---               │
│ f64               │
╞═══════════════════╡
│ 0.0676            │
└───────────────────┘


In [18]:
# 3. Actividad de los Anfitriones

reviews_quintiles = [
    airbnb_polars["reviews_per_month"].quantile(0.2),
    airbnb_polars["reviews_per_month"].quantile(0.4),
    airbnb_polars["reviews_per_month"].quantile(0.6),
    airbnb_polars["reviews_per_month"].quantile(0.8)
]


grouped_by_quintiles_polars = (
    airbnb_polars
    .with_columns(
        pl.when(pl.col("reviews_per_month") <= reviews_quintiles[0]).then(1)
        .when(pl.col("reviews_per_month") <= reviews_quintiles[1]).then(2)
        .when(pl.col("reviews_per_month") <= reviews_quintiles[2]).then(3)
        .when(pl.col("reviews_per_month") <= reviews_quintiles[3]).then(4)
        .otherwise(5).alias("reviews_quintile"),
    )
    .group_by("reviews_quintile")
    .agg(
        pl.col("availability_365").mean().round(2).alias("mean_availability"),
        pl.col("availability_365").count().alias("count")
    )
    .sort("reviews_quintile")
)

grouped_by_quintiles_polars

reviews_quintile,mean_availability,count
i32,f64,u32
1,113.52,4332
2,164.42,4212
3,175.34,4270
4,172.24,4217
5,156.32,9949


In [19]:
# 4. Tendencias Temporales de las Reseñas

time_since_last_review = (
    airbnb_spark
    .withColumn("days_since_last_review", sqlf.datediff(sqlf.current_date(), sqlf.col("last_review")))
    .groupBy("room_type")
    .agg(
        sqlf.round(sqlf.mean("days_since_last_review"), 0).alias("mean_days_since_last_review")
    )
    .orderBy("mean_days_since_last_review")
)

time_since_last_review.show()

+---------------+---------------------------+
|      room_type|mean_days_since_last_review|
+---------------+---------------------------+
|Entire home/apt|                      371.0|
|    Shared room|                      475.0|
|   Private room|                      569.0|
|     Hotel room|                      815.0|
+---------------+---------------------------+



In [20]:
# 4. Tendencias Temporales de las Reseñas

most_recent_reviews = (
    airbnb_spark
    .groupBy("neighbourhood")
    .agg(
        sqlf.sum("number_of_reviews_ltm").alias("total_reviews_ltm"),
        sqlf.count("*").alias("count_listings")
    )
    .orderBy("total_reviews_ltm", ascending=False)
    .limit(10)
)

most_recent_reviews.show()

+----------------+-----------------+--------------+
|   neighbourhood|total_reviews_ltm|count_listings|
+----------------+-----------------+--------------+
|     Embajadores|            52877|          2992|
|     Universidad|            41328|          2408|
|         Palacio|            34178|          1894|
|             Sol|            29790|          1431|
|          Cortes|            23664|          1108|
|        Justicia|            22206|          1332|
| Palos de Moguer|             9154|           451|
|      Guindalera|             5609|           422|
|        Pacífico|             5212|           331|
|Puerta del Angel|             4723|           385|
+----------------+-----------------+--------------+



In [21]:
# 4. Tendencias Temporales de las Reseñas

from datetime import date

today = date.today()

time_since_last_review_polars = (
    airbnb_polars
    .with_columns(
        (pl.lit(today) - pl.col("last_review")).alias("days_since_last_review")
    )
    .with_columns(
        pl.col("days_since_last_review").dt.total_days().alias("days_since_last_review")
    )
    .group_by("room_type")
    .agg(
        pl.col("days_since_last_review").mean().round(0).alias("mean_days_since_last_review")
    )
    .sort("mean_days_since_last_review")
)

print(time_since_last_review_polars)

shape: (4, 2)
┌─────────────────┬─────────────────────────────┐
│ room_type       ┆ mean_days_since_last_review │
│ ---             ┆ ---                         │
│ str             ┆ f64                         │
╞═════════════════╪═════════════════════════════╡
│ Entire home/apt ┆ 371.0                       │
│ Shared room     ┆ 475.0                       │
│ Private room    ┆ 569.0                       │
│ Hotel room      ┆ 815.0                       │
└─────────────────┴─────────────────────────────┘


In [22]:
# 4. Tendencias Temporales de las Reseñas

most_recent_reviews_polars = (
    airbnb_polars
    .group_by("neighbourhood")
    .agg(
        pl.col("number_of_reviews_ltm").sum().alias("total_reviews_ltm")
    )
    .sort("total_reviews_ltm", descending=True)
    .head(10)
)

most_recent_reviews_polars

neighbourhood,total_reviews_ltm
str,i64
"""Embajadores""",52877
"""Universidad""",41328
"""Palacio""",34178
"""Sol""",29790
"""Cortes""",23664
"""Justicia""",22206
"""Palos de Moguer""",9154
"""Guindalera""",5609
"""Pacífico""",5212


In [23]:
# 5. Duración Mínima de Estancia

min_nights_quintiles = airbnb_spark.approxQuantile("minimum_nights", [0.2, 0.4, 0.6, 0.8], 0.0)

price_by_minimum_nights = (
    airbnb_spark
    .withColumn(
        "min_nights_quintile",
        sqlf.when(sqlf.col("minimum_nights") <= min_nights_quintiles[0], 1)
        .when(sqlf.col("minimum_nights") <= min_nights_quintiles[1], 2)
        .when(sqlf.col("minimum_nights") <= min_nights_quintiles[2], 3)
        .when(sqlf.col("minimum_nights") <= min_nights_quintiles[3], 4)
        .otherwise(5)
    )
    .groupBy("min_nights_quintile")
    .agg(
        sqlf.round(sqlf.mean("price"), 2).alias("mean_price")
    )
    .orderBy("min_nights_quintile")
)

price_by_minimum_nights.show()

+-------------------+----------+
|min_nights_quintile|mean_price|
+-------------------+----------+
|                  1|    153.33|
|                  3|    124.27|
|                  4|    149.41|
|                  5|    117.06|
+-------------------+----------+



In [24]:
# 5. Duración Mínima de Estancia

min_nights_quintiles = [
    airbnb_polars["minimum_nights"].quantile(0.2),
    airbnb_polars["minimum_nights"].quantile(0.4),
    airbnb_polars["minimum_nights"].quantile(0.6),
    airbnb_polars["minimum_nights"].quantile(0.8)
]


price_by_minimum_nights_polars = (
    airbnb_polars
    .with_columns(
        pl.when(pl.col("minimum_nights") <= min_nights_quintiles[0]).then(1)
        .when(pl.col("minimum_nights") <= min_nights_quintiles[1]).then(2)
        .when(pl.col("minimum_nights") <= min_nights_quintiles[2]).then(3)
        .when(pl.col("minimum_nights") <= min_nights_quintiles[3]).then(4)
        .otherwise(5).alias("min_nights_quintile")
    )
    .group_by("min_nights_quintile")
    .agg(
        pl.col("price").mean().round(2).alias("mean_price")
    )
    .sort("min_nights_quintile")
)

price_by_minimum_nights_polars

min_nights_quintile,mean_price
i32,f64
1,153.33
3,124.27
4,149.41
5,117.06


In [25]:
# 5. Duración Mínima de Estancia

min_nights_by_room_type = (
    airbnb_spark
    .groupBy("room_type")
    .agg(
        sqlf.round(sqlf.mean("minimum_nights"), 2).alias("mean_minimum_nights")
    )
    .orderBy("mean_minimum_nights", ascending=False)
)

min_nights_by_room_type.show()

+---------------+-------------------+
|      room_type|mean_minimum_nights|
+---------------+-------------------+
|     Hotel room|              11.89|
|Entire home/apt|               8.48|
|   Private room|                7.4|
|    Shared room|                4.1|
+---------------+-------------------+



In [26]:
# 5. Duración Mínima de Estancia

min_nights_by_room_type_polars = (
    airbnb_polars
    .group_by("room_type")
    .agg([
        pl.col("minimum_nights").mean().round(2).alias("mean_minimum_nights")
    ])
    .sort("mean_minimum_nights", descending=True)
)

min_nights_by_room_type_polars

room_type,mean_minimum_nights
str,f64
"""Hotel room""",11.89
"""Entire home/apt""",8.48
"""Private room""",7.4
"""Shared room""",4.1


In [27]:
# 6. Popularidad de los Barrios

most_reviewed_neighbourhoods = (
    airbnb_spark
    .groupBy("neighbourhood")
    .agg(
        sqlf.sum("number_of_reviews").alias("total_reviews")
    )
    .orderBy(sqlf.desc("total_reviews"))
    .limit(10)
)

most_reviewed_neighbourhoods.show()


+---------------+-------------+
|  neighbourhood|total_reviews|
+---------------+-------------+
|    Embajadores|       192458|
|    Universidad|       151488|
|        Palacio|       127130|
|            Sol|       117131|
|         Cortes|        88061|
|       Justicia|        71621|
|Palos de Moguer|        28481|
|       Pacífico|        14382|
|      Trafalgar|        13599|
| Cuatro Caminos|        13038|
+---------------+-------------+



In [28]:
# 6. Popularidad de los Barrios

most_reviewed_neighbourhoods_polars = (
    airbnb_polars
    .group_by("neighbourhood")
    .agg(
        pl.col("number_of_reviews").sum().alias("total_reviews")
    )
    .sort("total_reviews", descending=True)
    .head(10)
)

most_reviewed_neighbourhoods_polars

neighbourhood,total_reviews
str,i64
"""Embajadores""",192458
"""Universidad""",151488
"""Palacio""",127130
"""Sol""",117131
"""Cortes""",88061
"""Justicia""",71621
"""Palos de Moguer""",28481
"""Pacífico""",14382
"""Trafalgar""",13599


In [29]:
# 6. Popularidad de los Barrios

most_reviewed_monthly_neighbourhoods = (
    airbnb_spark
    .groupBy("neighbourhood")
    .agg(
        sqlf.round(sqlf.mean("reviews_per_month"), 2).alias("avg_reviews_per_month")
    )
    .orderBy(sqlf.desc("avg_reviews_per_month"))
    .limit(10)
)

most_reviewed_monthly_neighbourhoods.show()

+--------------------+---------------------+
|       neighbourhood|avg_reviews_per_month|
+--------------------+---------------------+
|          Aeropuerto|                 6.22|
|               Timón|                 4.35|
|Casco Histórico d...|                 4.09|
|          Corralejos|                 3.83|
|    Alameda de Osuna|                 3.21|
|       Santa Eugenia|                  2.9|
|              Ambroz|                 2.63|
|           Entrevías|                 2.45|
|     Palos de Moguer|                 2.28|
|       Pinar del Rey|                 2.28|
+--------------------+---------------------+



In [74]:
# 6. Popularidad de los Barrios

most_reviewed_monthly_neighbourhoods_polars = (
    airbnb_polars
    .filter(pl.col("reviews_per_month").is_not_null())
    .group_by("neighbourhood")
    .agg(
        pl.col("reviews_per_month").mean().round(2).alias("avg_reviews_per_month")
    )
    .sort("avg_reviews_per_month", descending=True)
    .head(10)
)

most_reviewed_monthly_neighbourhoods_polars

neighbourhood,avg_reviews_per_month
str,f64
"""Aeropuerto""",6.22
"""Timón""",4.35
"""Casco Histórico de Barajas""",4.09
"""Corralejos""",3.83
"""Alameda de Osuna""",3.21
"""Santa Eugenia""",2.9
"""Ambroz""",2.63
"""Entrevías""",2.45
"""Palos de Moguer""",2.28


In [31]:
# 7. Impacto de la Actividad del Anfitrión

host_listings_quintiles = airbnb_spark.approxQuantile("calculated_host_listings_count", [0.2, 0.4, 0.6, 0.8], 0.0)

impact_by_host_activity = (
    airbnb_spark
    .withColumn(
        "host_listings_quintile",
        sqlf.when(sqlf.col("calculated_host_listings_count") <= host_listings_quintiles[0], 1)
        .when(sqlf.col("calculated_host_listings_count") <= host_listings_quintiles[1], 2)
        .when(sqlf.col("calculated_host_listings_count") <= host_listings_quintiles[2], 3)
        .when(sqlf.col("calculated_host_listings_count") <= host_listings_quintiles[3], 4)
        .otherwise(5)
    )
    .groupBy("host_listings_quintile")
    .agg(
        sqlf.round(sqlf.mean("price"), 2).alias("avg_price"),
        sqlf.round(sqlf.mean("reviews_per_month"), 2).alias("avg_reviews_per_month")
    )
    .orderBy("host_listings_quintile")
)

impact_by_host_activity.show()

+----------------------+---------+---------------------+
|host_listings_quintile|avg_price|avg_reviews_per_month|
+----------------------+---------+---------------------+
|                     1|   138.77|                 1.65|
|                     2|   121.63|                 2.03|
|                     3|   135.54|                 1.99|
|                     4|   152.39|                  2.1|
|                     5|   145.48|                 1.45|
+----------------------+---------+---------------------+



In [32]:
# 7. Impacto de la Actividad del Anfitrión

host_listings_quintiles = [
    airbnb_polars["calculated_host_listings_count"].quantile(0.2),
    airbnb_polars["calculated_host_listings_count"].quantile(0.4),
    airbnb_polars["calculated_host_listings_count"].quantile(0.6),
    airbnb_polars["calculated_host_listings_count"].quantile(0.8)
]


impact_by_host_activity_polars = (
    airbnb_polars
    .with_columns(
        pl.when(pl.col("calculated_host_listings_count") <= host_listings_quintiles[0]).then(1)
        .when(pl.col("calculated_host_listings_count") <= host_listings_quintiles[1]).then(2)
        .when(pl.col("calculated_host_listings_count") <= host_listings_quintiles[2]).then(3)
        .when(pl.col("calculated_host_listings_count") <= host_listings_quintiles[3]).then(4)
        .otherwise(5).alias("host_listings_quintile")
    )
    .group_by("host_listings_quintile")
    .agg(
        pl.col("price").mean().round(2).alias("avg_price"),
        pl.col("reviews_per_month").mean().round(2).alias("avg_reviews_per_month")
    )
    .sort("host_listings_quintile")
)

impact_by_host_activity_polars

host_listings_quintile,avg_price,avg_reviews_per_month
i32,f64,f64
1,138.77,1.65
2,121.63,2.03
3,135.54,1.99
4,152.39,2.1
5,145.48,1.45


In [33]:
# 7. Impacto de la Actividad del Anfitrión

top_hosts = (
    airbnb_spark
    .groupBy("host_id", "host_name")
    .agg(
        sqlf.sum("calculated_host_listings_count").alias("total_listings")
    )
    .orderBy(sqlf.desc("total_listings"))
    .limit(10)
)

top_hosts.show()

+---------+-------------------+--------------+
|  host_id|          host_name|total_listings|
+---------+-------------------+--------------+
|291253690|Fran Y Marta Isabel|        103684|
|346367515|               Ukio|         98596|
|377605855|          MIT House|         97344|
|518713670|           Bemadrid|         64516|
|434208714|         Blueground|         48841|
|438136382|          Home Club|         37249|
|576464572|     Maria Mercedes|         30976|
|444912851|               Duke|         17161|
|460391145|   Francisco Andres|         16900|
| 40877988|               Raul|         15376|
+---------+-------------------+--------------+



In [34]:
# 7. Impacto de la Actividad del Anfitrión

top_hosts_polars = (
    airbnb_polars
    .group_by("host_id", "host_name")
    .agg(
        pl.col("calculated_host_listings_count").sum().alias("total_listings")
    )
    .sort("total_listings", descending=True)
    .head(10)
)

top_hosts_polars

host_id,host_name,total_listings
i64,str,i64
291253690,"""Fran Y Marta Isabel""",103684
346367515,"""Ukio""",98596
377605855,"""MIT House""",97344
518713670,"""Bemadrid""",64516
434208714,"""Blueground""",48841
438136382,"""Home Club""",37249
576464572,"""Maria Mercedes""",30976
444912851,"""Duke""",17161
460391145,"""Francisco Andres""",16900


In [35]:
# 8. Licencias y Desempeño

license_counts = (
    airbnb_spark
    .groupBy((sqlf.col("license").isNotNull()).alias("has_license"))
    .agg(
        sqlf.count("*").alias("count"),
        sqlf.round((sqlf.count("*") / airbnb_spark.count()) * 100, 2).alias("percentage")
    )
)

license_counts.show()


+-----------+-----+----------+
|has_license|count|percentage|
+-----------+-----+----------+
|       true| 2630|      9.75|
|      false|24350|     90.25|
+-----------+-----+----------+



In [36]:
# 8. Licencias y Desempeño

license_counts_polars = (
    airbnb_polars
    .group_by(pl.col("license").is_not_null().alias("has_license"))
    .agg(
        pl.len().alias("count"),
        (pl.len() / airbnb_polars.shape[0] * 100).round(2).alias("percentage")
    )
)

license_counts_polars

has_license,count,percentage
bool,u32,f64
true,2630,9.75
false,24350,90.25


In [37]:
# 8. Licencias y Desempeño

performance_by_license = (
    airbnb_spark
    .groupBy((sqlf.col("license").isNotNull()).alias("has_license"))
    .agg(
        sqlf.round(sqlf.mean("price"), 2).alias("avg_price"),
        sqlf.round(sqlf.mean("availability_365"), 2).alias("avg_availability"),
        sqlf.round(sqlf.mean("reviews_per_month"), 2).alias("avg_reviews_per_month"),
        sqlf.count("*").alias("count")
    )
)

performance_by_license.show()

+-----------+---------+----------------+---------------------+-----+
|has_license|avg_price|avg_availability|avg_reviews_per_month|count|
+-----------+---------+----------------+---------------------+-----+
|       true|   182.47|          166.87|                 1.98| 2630|
|      false|   135.37|          155.06|                 1.78|24350|
+-----------+---------+----------------+---------------------+-----+



In [38]:
# 8. Licencias y Desempeño

performance_by_license_polars = (
    airbnb_polars
    .group_by(pl.col("license").is_not_null().alias("has_license"))
    .agg(
        pl.col("price").mean().round(2).alias("avg_price"),
        pl.col("availability_365").mean().round(2).alias("avg_availability"),
        pl.col("reviews_per_month").mean().round(2).alias("avg_reviews_per_month"),
        pl.len().alias("count")
    )
)

performance_by_license_polars

has_license,avg_price,avg_availability,avg_reviews_per_month,count
bool,f64,f64,f64,u32
false,135.37,155.06,1.78,24350
true,182.47,166.87,1.98,2630


In [39]:
# 9. Tendencias de Precios y Reseñas Recientes

correlations = (
    airbnb_spark.select(
        sqlf.round(sqlf.corr("number_of_reviews_ltm", "price") ,2).alias("corr_reviews_price"),
        sqlf.round(sqlf.corr("number_of_reviews_ltm", "availability_365") ,2).alias("corr_reviews_availability")
    )
)

correlations.show()

+------------------+-------------------------+
|corr_reviews_price|corr_reviews_availability|
+------------------+-------------------------+
|             -0.04|                     0.07|
+------------------+-------------------------+



In [40]:
# 9. Tendencias de Precios y Reseñas Recientes

corr_reviews_price = airbnb_polars.select(
    pl.corr("number_of_reviews_ltm", "price").round(2)
)

corr_reviews_availability = airbnb_polars.select(
    pl.corr("number_of_reviews_ltm", "availability_365").round(2)
)

print(f"Correlación entre reseñas y precio: {corr_reviews_price}")
print(f"Correlación entre reseñas y disponibilidad: {corr_reviews_availability}")

Correlación entre reseñas y precio: shape: (1, 1)
┌───────────────────────┐
│ number_of_reviews_ltm │
│ ---                   │
│ f64                   │
╞═══════════════════════╡
│ -0.04                 │
└───────────────────────┘
Correlación entre reseñas y disponibilidad: shape: (1, 1)
┌───────────────────────┐
│ number_of_reviews_ltm │
│ ---                   │
│ f64                   │
╞═══════════════════════╡
│ 0.07                  │
└───────────────────────┘


In [41]:
# 10. Correlación entre Características

correlations_price = (
    airbnb_spark.select(
        sqlf.round(sqlf.corr("minimum_nights", "price"),2),
        sqlf.round(sqlf.corr("number_of_reviews", "price"),2),
        sqlf.round(sqlf.corr("reviews_per_month", "price"),2),
        sqlf.round(sqlf.corr("number_of_reviews_ltm", "price"),2),
        sqlf.round(sqlf.corr("calculated_host_listings_count", "price"),2),
        sqlf.round(sqlf.corr("availability_365", "price"),2)

    )
)

correlations_availability_365 = (
    airbnb_spark.select(
        sqlf.round(sqlf.corr("minimum_nights", "availability_365"),2),
        sqlf.round(sqlf.corr("number_of_reviews", "availability_365"),2),
        sqlf.round(sqlf.corr("reviews_per_month", "availability_365"),2),
        sqlf.round(sqlf.corr("number_of_reviews_ltm", "availability_365"),2),
        sqlf.round(sqlf.corr("calculated_host_listings_count", "availability_365"),2),
        sqlf.round(sqlf.corr("price", "availability_365"),2)

    )
)

correlations_price.show()
correlations_availability_365.show()

+-------------------------------------+----------------------------------------+----------------------------------------+--------------------------------------------+-----------------------------------------------------+---------------------------------------+
|round(corr(minimum_nights, price), 2)|round(corr(number_of_reviews, price), 2)|round(corr(reviews_per_month, price), 2)|round(corr(number_of_reviews_ltm, price), 2)|round(corr(calculated_host_listings_count, price), 2)|round(corr(availability_365, price), 2)|
+-------------------------------------+----------------------------------------+----------------------------------------+--------------------------------------------+-----------------------------------------------------+---------------------------------------+
|                                  0.0|                                   -0.02|                                   -0.04|                                       -0.04|                                                 0.

In [42]:
# 10. Correlación entre Características

corr_reviews_price = airbnb_polars.select(
    pl.corr("number_of_reviews_ltm", "price").round(2),
    pl.corr("availability_365", "price").round(2),
    pl.corr("minimum_nights", "price").round(2),
    pl.corr("reviews_per_month", "price").round(2),
    pl.corr("calculated_host_listings_count", "price").round(2),
    pl.corr("number_of_reviews", "price").round(2)
)

corr_reviews_availability_365 = airbnb_polars.select(
    pl.corr("number_of_reviews_ltm", "availability_365").round(2),
    pl.corr("price", "availability_365").round(2),
    pl.corr("minimum_nights", "availability_365").round(2),
    pl.corr("reviews_per_month", "availability_365").round(2),
    pl.corr("calculated_host_listings_count", "availability_365").round(2),
    pl.corr("number_of_reviews", "availability_365").round(2)
)

corr_reviews_price, corr_reviews_availability_365


(shape: (1, 6)
 ┌────────────────┬────────────────┬────────────────┬───────────────┬───────────────┬───────────────┐
 │ number_of_revi ┆ availability_3 ┆ minimum_nights ┆ reviews_per_m ┆ calculated_ho ┆ number_of_rev │
 │ ews_ltm        ┆ 65             ┆ ---            ┆ onth          ┆ st_listings_c ┆ iews          │
 │ ---            ┆ ---            ┆ f64            ┆ ---           ┆ ount          ┆ ---           │
 │ f64            ┆ f64            ┆                ┆ f64           ┆ ---           ┆ f64           │
 │                ┆                ┆                ┆               ┆ f64           ┆               │
 ╞════════════════╪════════════════╪════════════════╪═══════════════╪═══════════════╪═══════════════╡
 │ -0.04          ┆ 0.02           ┆ -0.0           ┆ -0.04         ┆ 0.01          ┆ -0.02         │
 └────────────────┴────────────────┴────────────────┴───────────────┴───────────────┴───────────────┘,
 shape: (1, 6)
 ┌──────────────────┬───────┬────────────────┬─────

In [43]:
# 11. Demanda según Tipo de Habitación

room_demand_spark = (
    airbnb_spark.groupBy("room_type")
    .agg(sqlf.round(sqlf.avg("reviews_per_month"), 2).alias("avg_reviews_per_month"))
    .orderBy(sqlf.desc("avg_reviews_per_month"))
)

room_demand_spark.show()

+---------------+---------------------+
|      room_type|avg_reviews_per_month|
+---------------+---------------------+
|Entire home/apt|                 1.92|
|    Shared room|                 1.74|
|   Private room|                 1.57|
|     Hotel room|                 0.47|
+---------------+---------------------+



In [44]:
# 11. Demanda según Tipo de Habitación

room_demand_polars = (
    airbnb_polars
    .group_by("room_type")
    .agg(pl.col("reviews_per_month").mean().round(2).alias("avg_reviews_per_month"))
    .sort("avg_reviews_per_month", descending=True)
)

room_demand_polars

room_type,avg_reviews_per_month
str,f64
"""Entire home/apt""",1.92
"""Shared room""",1.74
"""Private room""",1.57
"""Hotel room""",0.47


In [45]:
# 11. Demanda según Tipo de Habitación

price_distribution_spark = (
    airbnb_spark.groupBy("room_type")
    .agg(
        sqlf.count("*").alias("count"),
        sqlf.round(sqlf.mean("price"), 2).alias("mean_price"),
        sqlf.min("price").alias("min_price"),
        sqlf.max("price").alias("max_price"),
        sqlf.percentile("price", 0.5).alias("median_price")
    )
)

price_distribution_spark.show()

+---------------+-----+----------+---------+---------+------------+
|      room_type|count|mean_price|min_price|max_price|median_price|
+---------------+-----+----------+---------+---------+------------+
|    Shared room|  343|    100.05|     13.0|   1500.0|        37.0|
|     Hotel room|   92|    303.66|     31.0|   1029.0|       161.0|
|Entire home/apt|17490|    169.14|      1.0|  21000.0|       124.0|
|   Private room| 9055|     72.76|      8.0|   9218.0|        45.0|
+---------------+-----+----------+---------+---------+------------+



In [46]:
# 11. Demanda según Tipo de Habitación

price_distribution_polars = (
    airbnb_polars.group_by("room_type")
    .agg(
        pl.col("price").count().alias("count"),
        pl.col("price").mean().round(2).alias("mean_price"),
        pl.col("price").min().alias("min_price"),
        pl.col("price").max().alias("max_price"),
        pl.col("price").median().alias("median_price")
    )
)


price_distribution_polars

room_type,count,mean_price,min_price,max_price,median_price
str,u32,f64,f64,f64,f64
"""Shared room""",283,100.05,13.0,1500.0,37.0
"""Private room""",6110,72.76,8.0,9218.0,45.0
"""Hotel room""",53,303.66,31.0,1029.0,161.0
"""Entire home/apt""",14622,169.14,1.0,21000.0,124.0


# Datos precio alquiler metro cuadrado (polars solo)

In [47]:
madrid_df = pl.read_csv(DATA_DIR + "/0504030000213.csv", skip_rows= 4, null_values= "..")
madrid_df = madrid_df.rename({"":"year", "_duplicated_0":"neighbourhood"})
madrid_df = (
    madrid_df
    .with_columns(
        pl.col("neighbourhood").str.replace("\d+\.", "").str.strip_chars(),
        *[pl.col(col).str.replace(",", ".").cast(pl.Float64) for col in madrid_df.columns[2:]]

    )
)

madrid_df.head()

year,neighbourhood,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2006,"""Ciudad de Madrid""",null,null,null,null,null,null,null,null,null,null,null,null
2006,"""Centro""",null,null,null,null,null,null,null,null,null,null,null,null
2006,"""Arganzuela""",null,null,null,null,null,null,null,null,null,null,null,null
2006,"""Retiro""",null,null,null,null,null,null,null,null,null,null,null,null
2006,"""Salamanca""",null,null,null,null,null,null,null,null,null,null,null,null


In [48]:
madrid_df.tail()

year,neighbourhood,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024,"""Villaverde""",13.16,13.7,14.08,13.93,14.12,14.38,15.0,15.25,15.83,15.79,null,null
2024,"""Villa de Vallecas""",13.25,13.57,14.01,14.3,14.55,14.27,14.25,14.35,14.68,14.98,null,null
2024,"""Vicálvaro""",12.8,12.86,13.22,13.08,12.96,12.95,13.5,13.82,13.81,13.9,null,null
2024,"""San Blas-Canillejas""",13.92,13.99,14.12,14.54,14.6,14.97,14.94,14.77,14.79,15.18,null,null
2024,"""Barajas""",14.11,13.95,13.92,14.08,14.5,14.91,14.69,14.6,14.6,14.78,null,null


In [49]:
# Melt the DataFrame
melted_df = madrid_df.unpivot(
    index=["year", "neighbourhood"],  # Columns to keep as is
    on=[
        "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
        "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
    ],  # Columns to melt
    variable_name="month",  # New column name for month
    value_name="price"  # New column name for prices
)

In [50]:
melted_df.sample(10)

year,neighbourhood,month,price
i64,str,str,f64
2018,"""Puente de Vallecas""","""Agosto""",11.2
2023,"""San Blas-Canillejas""","""Febrero""",12.66
2015,"""Carabanchel""","""Febrero""",8.5
2015,"""Usera""","""Abril""",8.5
2008,"""Latina""","""Febrero""",null
2021,"""Moratalaz""","""Julio""",11.0
2020,"""San Blas-Canillejas""","""Julio""",12.2
2016,"""Centro""","""Mayo""",15.7
2022,"""Hortaleza""","""Diciembre""",13.3


In [51]:
# Month-to-integer mapping
month_to_int = {
    "Enero": 1,
    "Febrero": 2,
    "Marzo": 3,
    "Abril": 4,
    "Mayo": 5,
    "Junio": 6,
    "Julio": 7,
    "Agosto": 8,
    "Septiembre": 9,
    "Octubre": 10,
    "Noviembre": 11,
    "Diciembre": 12,
}

# Map months to integers and create the year_month column
processed_df = melted_df.with_columns(
    pl.col("month").replace(month_to_int).alias("month_int")
).with_columns(
    (pl.col("year").cast(str) + "-" + pl.col("month_int").cast(str).str.zfill(2)).alias("year_month")
).drop_nulls()

processed_df.sample(10)

year,neighbourhood,month,price,month_int,year_month
i64,str,str,f64,str,str
2016,"""Chamberí""","""Mayo""",15.0,"""5""","""2016-05"""
2018,"""Retiro""","""Marzo""",15.4,"""3""","""2018-03"""
2009,"""Salamanca""","""Junio""",16.0,"""6""","""2009-06"""
2013,"""Salamanca""","""Mayo""",13.6,"""5""","""2013-05"""
2021,"""Villaverde""","""Diciembre""",10.9,"""12""","""2021-12"""
2016,"""Latina""","""Noviembre""",9.3,"""11""","""2016-11"""
2022,"""San Blas-Canillejas""","""Septiembre""",12.3,"""9""","""2022-09"""
2016,"""Fuencarral-El Pardo""","""Enero""",10.3,"""1""","""2016-01"""
2011,"""Moncloa-Aravaca""","""Julio""",12.3,"""7""","""2011-07"""


In [52]:
# Pivot the table to create a column for each barrio
rent_price_df = processed_df.pivot(
    values="price",  # Values to pivot
    index="year_month",  # Rows are unique combinations of year and month
    on="neighbourhood",  # Columns are unique barrios
    aggregate_function=None  # No aggregation needed since data is already unique
).sort('year_month')

rent_price_df.sample(10)

year_month,Ciudad de Madrid,Centro,Arganzuela,Salamanca,Chamartín,Tetuán,Chamberí,Fuencarral-El Pardo,Moncloa-Aravaca,Carabanchel,Ciudad Lineal,Hortaleza,San Blas-Canillejas,Retiro,Latina,Usera,Puente de Vallecas,Villa de Vallecas,Barajas,Moratalaz,Villaverde,Vicálvaro
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2020-05""",16.8,16.8,16.4,19.3,16.6,16.4,18.5,12.9,15.0,12.6,14.0,13.0,12.1,16.8,13.0,12.5,12.6,11.3,11.5,12.0,11.6,11.1
"""2011-10""",12.4,14.9,11.9,14.6,13.3,12.5,14.3,10.8,12.3,9.7,11.1,11.5,10.4,12.6,9.5,9.3,9.9,10.0,10.8,9.2,8.6,9.0
"""2010-01""",12.8,16.3,12.5,15.2,14.3,13.8,14.6,9.9,13.3,10.1,11.3,10.9,11.8,null,null,null,null,null,null,null,null,null
"""2009-03""",13.0,17.5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""2020-11""",15.4,17.1,14.8,17.3,15.8,15.0,16.8,12.5,14.8,11.7,13.2,12.3,11.9,15.3,12.2,11.5,12.3,11.5,11.4,11.2,10.8,10.2
"""2012-06""",11.9,14.2,11.7,14.0,12.9,12.3,14.1,10.3,11.6,9.6,10.9,11.0,10.2,12.5,9.2,9.0,9.6,9.0,10.3,9.0,8.5,8.5
"""2021-06""",14.5,16.2,14.2,16.7,15.0,14.3,16.3,12.1,14.3,11.4,12.7,12.6,11.4,14.8,11.5,10.9,11.6,11.1,11.4,11.1,10.4,10.2
"""2012-09""",11.9,14.0,11.5,14.1,12.5,12.0,13.9,10.3,11.5,9.2,10.7,10.7,9.9,12.2,9.3,8.6,9.3,9.2,10.3,8.7,8.5,8.4
"""2024-08""",19.92,23.61,19.88,24.45,20.44,20.0,23.03,16.1,19.43,15.82,17.0,15.79,14.77,21.04,16.29,16.55,16.16,14.35,14.6,14.49,15.25,13.82


In [53]:
oct2024_dicc = rent_price_df.filter(
    pl.col("year_month") == "2024-10"
).to_dicts()[0]

oct2011_dicc = rent_price_df.filter(
    pl.col("year_month") == "2011-10"
).to_dicts()[0]


In [54]:
oct2024_dicc, oct2011_dicc


({'year_month': '2024-10',
  'Ciudad de Madrid': 20.61,
  'Centro': 24.44,
  'Arganzuela': 20.45,
  'Salamanca': 24.99,
  'Chamartín': 20.88,
  'Tetuán': 20.42,
  'Chamberí': 24.14,
  'Fuencarral-El Pardo': 16.09,
  'Moncloa-Aravaca': 19.79,
  'Carabanchel': 16.11,
  'Ciudad Lineal': 17.22,
  'Hortaleza': 16.18,
  'San Blas-Canillejas': 15.18,
  'Retiro': 21.45,
  'Latina': 17.04,
  'Usera': 17.11,
  'Puente de Vallecas': 16.25,
  'Villa de Vallecas': 14.98,
  'Barajas': 14.78,
  'Moratalaz': 14.67,
  'Villaverde': 15.79,
  'Vicálvaro': 13.9},
 {'year_month': '2011-10',
  'Ciudad de Madrid': 12.4,
  'Centro': 14.9,
  'Arganzuela': 11.9,
  'Salamanca': 14.6,
  'Chamartín': 13.3,
  'Tetuán': 12.5,
  'Chamberí': 14.3,
  'Fuencarral-El Pardo': 10.8,
  'Moncloa-Aravaca': 12.3,
  'Carabanchel': 9.7,
  'Ciudad Lineal': 11.1,
  'Hortaleza': 11.5,
  'San Blas-Canillejas': 10.4,
  'Retiro': 12.6,
  'Latina': 9.5,
  'Usera': 9.3,
  'Puente de Vallecas': 9.9,
  'Villa de Vallecas': 10.0,
  'Baraja

In [55]:
for key in oct2024_dicc:
  if key != "year_month":
    quotient = round(oct2024_dicc[key]/oct2011_dicc[key], 2)
    print(key, quotient)


Ciudad de Madrid 1.66
Centro 1.64
Arganzuela 1.72
Salamanca 1.71
Chamartín 1.57
Tetuán 1.63
Chamberí 1.69
Fuencarral-El Pardo 1.49
Moncloa-Aravaca 1.61
Carabanchel 1.66
Ciudad Lineal 1.55
Hortaleza 1.41
San Blas-Canillejas 1.46
Retiro 1.7
Latina 1.79
Usera 1.84
Puente de Vallecas 1.64
Villa de Vallecas 1.5
Barajas 1.37
Moratalaz 1.59
Villaverde 1.84
Vicálvaro 1.54


In [56]:

cocientes = [
    oct2024_dicc[neighbourhood]/oct2011_dicc[neighbourhood]
    for neighbourhood in oct2024_dicc
    if neighbourhood != "year_month" and oct2011_dicc[neighbourhood] != 0
]

media_subida = round(pl.Series(cocientes).mean(), 2)
mediana_subida = round(pl.Series(cocientes).median(), 2)

print(f"Media de subida: {media_subida}")
print(f"Mediana de subida: {mediana_subida}")

Media de subida: 1.62
Mediana de subida: 1.64


In [57]:
ine_df = pl.read_csv(DATA_DIR + "/10882.csv", separator=";", encoding="ISO-8859-1")

ine_df = ine_df.with_columns(
    pl.col("Total")
        .str.replace(r"\.", "", literal=False)
        .str.replace(",", ".")
        .cast(pl.Float64)
)


salario_mediano_df = ine_df.filter(pl.col("Salario anual") == "Salario mediano")
salarios_agrupados = (
    salario_mediano_df
    .filter(pl.col("Periodo").is_in([2011, 2021]))
    .group_by("Periodo")
    .agg(pl.col("Total").mean().alias("salario_mediano"))
    .sort("Periodo")
)
print(salarios_agrupados)


price_2011 = rent_price_df.filter(pl.col("year_month") == "2011-10").select("Ciudad de Madrid").item()
price_2021 = rent_price_df.filter(pl.col("year_month") == "2021-10").select("Ciudad de Madrid").item()

print(f"Precio €/m² en 2011-10: {price_2011}")
print(f"Precio €/m² en 2021-10: {price_2021}")


alquiler_anual_2011 = price_2011 * 85 * 12
alquiler_anual_2021 = price_2021 * 85 * 12

print(f"Alquiler anual 2011: {alquiler_anual_2011:.2f} €")
print(f"Alquiler anual 2021: {alquiler_anual_2021:.2f} €")


salarios_dict = {row["Periodo"]: row["salario_mediano"] for row in salarios_agrupados.to_dicts()}

porcentaje_2011 = (alquiler_anual_2011 / salarios_dict[2011]) * 100
porcentaje_2021 = (alquiler_anual_2021 / salarios_dict[2021]) * 100

print(f"En 2011, el alquiler anual de una vivienda de 85 m² representaba el {porcentaje_2011:.2f}% del salario mediano bruto.")
print(f"En 2021, el alquiler anual de una vivienda de 85 m² representaba el {porcentaje_2021:.2f}% del salario mediano bruto.")


shape: (2, 2)
┌─────────┬─────────────────┐
│ Periodo ┆ salario_mediano │
│ ---     ┆ ---             │
│ i64     ┆ f64             │
╞═════════╪═════════════════╡
│ 2011    ┆ 19142.2         │
│ 2021    ┆ 21326.545       │
└─────────┴─────────────────┘
Precio €/m² en 2011-10: 12.4
Precio €/m² en 2021-10: 14.8
Alquiler anual 2011: 12648.00 €
Alquiler anual 2021: 15096.00 €
En 2011, el alquiler anual de una vivienda de 85 m² representaba el 66.07% del salario mediano bruto.
En 2021, el alquiler anual de una vivienda de 85 m² representaba el 70.79% del salario mediano bruto.


In [58]:
salario_2011 = {
    row["Sexo/Brecha de género"]: row["Total"]
    for row in salario_mediano_df.filter(pl.col("Periodo") == 2011).to_dicts()
}
salario_2021 = {
    row["Sexo/Brecha de género"]: row["Total"]
    for row in salario_mediano_df.filter(pl.col("Periodo") == 2021).to_dicts()
}


In [59]:
# Calculamos el incremento (cociente) de salarios medianos para cada género
for genero in salario_2011.keys():
    incremento_salario = ((salario_2021[genero] / salario_2011[genero]) * 100) - 100
    print(f"Para {genero}, el salario mediano ha subido un {incremento_salario:.2f}%")


Para Mujeres, el salario mediano ha subido un 14.09%
Para Hombres, el salario mediano ha subido un 9.32%


In [60]:
# Se genera un DataFrame con el incremento por barrio.
rent_increase_data = []
for col in rent_price_df.columns:
    if col != "year_month":

        val_2011 = rent_price_df.filter(pl.col("year_month") == "2011-10").select(col).item()
        val_2024 = rent_price_df.filter(pl.col("year_month") == "2024-10").select(col).item()

        rent_increase_data.append({
            "neighbourhood": col.lower(),
            "rent_increase": round((val_2024 / val_2011), 2)
        })

rent_increase_df = pl.DataFrame(rent_increase_data)
rent_increase_df

neighbourhood,rent_increase
str,f64
"""ciudad de madrid""",1.66
"""centro""",1.64
"""arganzuela""",1.72
"""salamanca""",1.71
"""chamartín""",1.57
…,…
"""villa de vallecas""",1.5
"""barajas""",1.37
"""moratalaz""",1.59


In [61]:
# Normalizar la columna "neighbourhood" en airbnb_df
airbnb_polars_new = airbnb_polars.with_columns(
    pl.col("neighbourhood")
      .str.to_lowercase()
      .str.replace_all(r"[^\w\s]", "")  # Eliminar símbolos que puedan interferir
      .alias("neighbourhood_norm")
)

# Agrupar por barrio: contar el número de anuncios y calcular el precio medio.
airbnb_grouped = (
    airbnb_polars_new
    .group_by("neighbourhood_norm")
    .agg([
        pl.count("id").alias("airbnb_count"),
        pl.col("price").mean().round(2).alias("airbnb_price_mean")
    ])
)
airbnb_grouped

neighbourhood_norm,airbnb_count,airbnb_price_mean
str,u32,f64
"""delicias""",241,88.86
"""berruguete""",204,105.02
"""rosas""",78,190.63
"""san fermín""",67,76.53
"""arapiles""",275,118.83
…,…,…
"""almendrales""",114,74.62
"""corralejos""",17,69.85
"""rios rosas""",266,154.95


In [62]:
airbnb_grouped, rent_increase_df

(shape: (128, 3)
 ┌────────────────────┬──────────────┬───────────────────┐
 │ neighbourhood_norm ┆ airbnb_count ┆ airbnb_price_mean │
 │ ---                ┆ ---          ┆ ---               │
 │ str                ┆ u32          ┆ f64               │
 ╞════════════════════╪══════════════╪═══════════════════╡
 │ delicias           ┆ 241          ┆ 88.86             │
 │ berruguete         ┆ 204          ┆ 105.02            │
 │ rosas              ┆ 78           ┆ 190.63            │
 │ san fermín         ┆ 67           ┆ 76.53             │
 │ arapiles           ┆ 275          ┆ 118.83            │
 │ …                  ┆ …            ┆ …                 │
 │ almendrales        ┆ 114          ┆ 74.62             │
 │ corralejos         ┆ 17           ┆ 69.85             │
 │ rios rosas         ┆ 266          ┆ 154.95            │
 │ casa de campo      ┆ 87           ┆ 121.75            │
 │ el viso            ┆ 97           ┆ 167.46            │
 └────────────────────┴──────────────┴─

In [63]:
audio_df = pl.read_parquet(DATA_DIR + "/fragments_example.parquet")

audio_df.shape, audio_df.schema, audio_df.head()

((194, 5),
 Schema([('audio_id', String),
         ('end', Int64),
         ('side', String),
         ('start', Int64),
         ('timestamps', List(Int64))]),
 shape: (5, 5)
 ┌────────────┬───────┬──────────┬───────┬─────────────────────────┐
 │ audio_id   ┆ end   ┆ side     ┆ start ┆ timestamps              │
 │ ---        ┆ ---   ┆ ---      ┆ ---   ┆ ---                     │
 │ str        ┆ i64   ┆ str      ┆ i64   ┆ list[i64]               │
 ╞════════════╪═══════╪══════════╪═══════╪═════════════════════════╡
 │ a4277c9fac ┆ 5906  ┆ client   ┆ 2128  ┆ [2668, 2968, … 5766]    │
 │ a4277c9fac ┆ 7906  ┆ operator ┆ 5906  ┆ [6146, 6506, … 7796]    │
 │ a4277c9fac ┆ 8905  ┆ client   ┆ 7906  ┆ [8356]                  │
 │ a4277c9fac ┆ 12906 ┆ operator ┆ 8906  ┆ [9386, 9776, … 12896]   │
 │ a4277c9fac ┆ 14784 ┆ client   ┆ 12906 ┆ [13566, 13716, … 14646] │
 └────────────┴───────┴──────────┴───────┴─────────────────────────┘)

In [64]:
# Ordenar la tabla por audio_id, start y end

audio_df = audio_df.sort(["audio_id", "start", "end"])

# Cálculos para cada fragmento

In [65]:
# 1. Calcular n_intervention

audio_df = audio_df.with_columns(
    (pl.int_range(0, pl.len()).over("audio_id") // 2).alias("n_intervention")
)

audio_df.head(30)

audio_id,end,side,start,timestamps,n_intervention
str,i64,str,i64,list[i64],i64
"""0871505d87""",4368,"""client""",1408,"[1708, 4208]",0
"""0871505d87""",9136,"""operator""",6624,"[7134, 7374, … 9084]",0
"""0871505d87""",10496,"""client""",9616,"[9946, 10126, 10396]",1
"""0871505d87""",22816,"""operator""",10864,"[11194, 11314, … 22804]",1
"""0871505d87""",30448,"""client""",23776,"[24016, 24226, … 30376]",2
…,…,…,…,…,…
"""3b0eaeb9f5""",75642,"""operator""",74142,"[74352, 75102, 75462]",5
"""3b0eaeb9f5""",91184,"""client""",75642,"[76002, 76512, … 91064]",5
"""3b0eaeb9f5""",93446,"""operator""",91696,"[92056, 92326, … 93376]",6


In [66]:
# 2. Calcular word_duration:

audio_df = audio_df.with_columns(
    pl.concat_list([pl.col("start"), pl.col("timestamps")])
    .list.diff()
    .list.slice(1)
    .list.eval(pl.element().clip(50, 1500))
    .alias("word_duration")
)

audio_df.head(30)

audio_id,end,side,start,timestamps,n_intervention,word_duration
str,i64,str,i64,list[i64],i64,list[i64]
"""0871505d87""",4368,"""client""",1408,"[1708, 4208]",0,"[300, 1500]"
"""0871505d87""",9136,"""operator""",6624,"[7134, 7374, … 9084]",0,"[510, 240, … 480]"
"""0871505d87""",10496,"""client""",9616,"[9946, 10126, 10396]",1,"[330, 180, 270]"
"""0871505d87""",22816,"""operator""",10864,"[11194, 11314, … 22804]",1,"[330, 120, … 480]"
"""0871505d87""",30448,"""client""",23776,"[24016, 24226, … 30376]",2,"[240, 210, … 420]"
…,…,…,…,…,…,…
"""3b0eaeb9f5""",75642,"""operator""",74142,"[74352, 75102, 75462]",5,"[210, 750, 360]"
"""3b0eaeb9f5""",91184,"""client""",75642,"[76002, 76512, … 91064]",5,"[360, 510, … 360]"
"""3b0eaeb9f5""",93446,"""operator""",91696,"[92056, 92326, … 93376]",6,"[360, 270, … 300]"


In [67]:
# 3. Calcular wait_silence:

audio_df = audio_df.with_columns(
    pl.when(pl.col("end").shift(1).over("audio_id").is_null())
      .then(pl.col("start"))
      .otherwise(pl.col("start") - pl.col("end").shift(1).over("audio_id"))
      .alias("wait_silence")
)

audio_df.head(30)

audio_id,end,side,start,timestamps,n_intervention,word_duration,wait_silence
str,i64,str,i64,list[i64],i64,list[i64],i64
"""0871505d87""",4368,"""client""",1408,"[1708, 4208]",0,"[300, 1500]",1408
"""0871505d87""",9136,"""operator""",6624,"[7134, 7374, … 9084]",0,"[510, 240, … 480]",2256
"""0871505d87""",10496,"""client""",9616,"[9946, 10126, 10396]",1,"[330, 180, 270]",480
"""0871505d87""",22816,"""operator""",10864,"[11194, 11314, … 22804]",1,"[330, 120, … 480]",368
"""0871505d87""",30448,"""client""",23776,"[24016, 24226, … 30376]",2,"[240, 210, … 420]",960
…,…,…,…,…,…,…,…
"""3b0eaeb9f5""",75642,"""operator""",74142,"[74352, 75102, 75462]",5,"[210, 750, 360]",0
"""3b0eaeb9f5""",91184,"""client""",75642,"[76002, 76512, … 91064]",5,"[360, 510, … 360]",0
"""3b0eaeb9f5""",93446,"""operator""",91696,"[92056, 92326, … 93376]",6,"[360, 270, … 300]",512


In [68]:
audio_df = audio_df.with_columns(
    ((pl.col('start') - pl.col('end').shift(1)).fill_null(pl.col('start').first())
                 .over('audio_id') # lo hacemos por audio_id para no mezclar fragmentos de distintos audios
                 .clip(0)
                ).alias('wait_silence_2')
)

audio_df.head(30)


audio_id,end,side,start,timestamps,n_intervention,word_duration,wait_silence,wait_silence_2
str,i64,str,i64,list[i64],i64,list[i64],i64,i64
"""0871505d87""",4368,"""client""",1408,"[1708, 4208]",0,"[300, 1500]",1408,1408
"""0871505d87""",9136,"""operator""",6624,"[7134, 7374, … 9084]",0,"[510, 240, … 480]",2256,2256
"""0871505d87""",10496,"""client""",9616,"[9946, 10126, 10396]",1,"[330, 180, 270]",480,480
"""0871505d87""",22816,"""operator""",10864,"[11194, 11314, … 22804]",1,"[330, 120, … 480]",368,368
"""0871505d87""",30448,"""client""",23776,"[24016, 24226, … 30376]",2,"[240, 210, … 420]",960,960
…,…,…,…,…,…,…,…,…
"""3b0eaeb9f5""",75642,"""operator""",74142,"[74352, 75102, 75462]",5,"[210, 750, 360]",0,0
"""3b0eaeb9f5""",91184,"""client""",75642,"[76002, 76512, … 91064]",5,"[360, 510, … 360]",0,0
"""3b0eaeb9f5""",93446,"""operator""",91696,"[92056, 92326, … 93376]",6,"[360, 270, … 300]",512,512


In [69]:
audio_df.select(
    (pl.col("wait_silence") == pl.col("wait_silence_2")).all()
)

wait_silence
bool
true


In [70]:
# 4. Calcular sum_silence (a nivel fragmento):

audio_df = audio_df.with_columns(
    pl.col("word_duration")
      .list.eval(
          pl.when(pl.element() > 350)
            .then(pl.element() - 350)
            .otherwise(0)
      )
      .list.sum()
      .alias("sum_silence")
)

audio_df.head(30)

audio_id,end,side,start,timestamps,n_intervention,word_duration,wait_silence,wait_silence_2,sum_silence
str,i64,str,i64,list[i64],i64,list[i64],i64,i64,i64
"""0871505d87""",4368,"""client""",1408,"[1708, 4208]",0,"[300, 1500]",1408,1408,1150
"""0871505d87""",9136,"""operator""",6624,"[7134, 7374, … 9084]",0,"[510, 240, … 480]",2256,2256,340
"""0871505d87""",10496,"""client""",9616,"[9946, 10126, 10396]",1,"[330, 180, 270]",480,480,0
"""0871505d87""",22816,"""operator""",10864,"[11194, 11314, … 22804]",1,"[330, 120, … 480]",368,368,2030
"""0871505d87""",30448,"""client""",23776,"[24016, 24226, … 30376]",2,"[240, 210, … 420]",960,960,340
…,…,…,…,…,…,…,…,…,…
"""3b0eaeb9f5""",75642,"""operator""",74142,"[74352, 75102, 75462]",5,"[210, 750, 360]",0,0,410
"""3b0eaeb9f5""",91184,"""client""",75642,"[76002, 76512, … 91064]",5,"[360, 510, … 360]",0,0,2742
"""3b0eaeb9f5""",93446,"""operator""",91696,"[92056, 92326, … 93376]",6,"[360, 270, … 300]",512,512,10


In [71]:
audio_df = audio_df.with_columns(
    pl.col("word_duration")
      .list.eval(
          (pl.element() - 350).clip(0)
      )
      .list.sum()
      .alias("sum_silence_2")
)

audio_df.head(30)

audio_id,end,side,start,timestamps,n_intervention,word_duration,wait_silence,wait_silence_2,sum_silence,sum_silence_2
str,i64,str,i64,list[i64],i64,list[i64],i64,i64,i64,i64
"""0871505d87""",4368,"""client""",1408,"[1708, 4208]",0,"[300, 1500]",1408,1408,1150,1150
"""0871505d87""",9136,"""operator""",6624,"[7134, 7374, … 9084]",0,"[510, 240, … 480]",2256,2256,340,340
"""0871505d87""",10496,"""client""",9616,"[9946, 10126, 10396]",1,"[330, 180, 270]",480,480,0,0
"""0871505d87""",22816,"""operator""",10864,"[11194, 11314, … 22804]",1,"[330, 120, … 480]",368,368,2030,2030
"""0871505d87""",30448,"""client""",23776,"[24016, 24226, … 30376]",2,"[240, 210, … 420]",960,960,340,340
…,…,…,…,…,…,…,…,…,…,…
"""3b0eaeb9f5""",75642,"""operator""",74142,"[74352, 75102, 75462]",5,"[210, 750, 360]",0,0,410,410
"""3b0eaeb9f5""",91184,"""client""",75642,"[76002, 76512, … 91064]",5,"[360, 510, … 360]",0,0,2742,2742
"""3b0eaeb9f5""",93446,"""operator""",91696,"[92056, 92326, … 93376]",6,"[360, 270, … 300]",512,512,10,10


In [72]:
audio_df.select(
    (pl.col("sum_silence") == pl.col("sum_silence_2")).all()
)

sum_silence
bool
true


# Agregando para cada llamada y side (audio_id y side)

In [73]:

agg_df = audio_df.group_by("audio_id", "side").agg(
    pl.col("sum_silence").sum().alias("sum_silence"),
    pl.col("start").min().alias("first_silence"),
    (pl.col("timestamps").list.len().sum() / ((pl.col("end") - pl.col("start")).sum() / 1000))
    .round(2)
    .alias("word_speed")
)

agg_df.head(30)

audio_id,side,sum_silence,first_silence,word_speed
str,str,i64,i64,f64
"""0871505d87""","""operator""",7360,6624,4.17
"""574a49aa58""","""operator""",7448,1344,4.15
"""82cec0fd50""","""operator""",15994,1840,3.07
"""4693369d1e""","""operator""",16126,2272,3.35
"""a4277c9fac""","""client""",18144,2128,1.85
…,…,…,…,…
"""574a49aa58""","""client""",9912,528,3.31
"""6a8fe11f45""","""client""",7520,1520,3.02
"""4693369d1e""","""client""",5160,1104,2.52
